In [ ]:
import numpy as np
from copy import deepcopy
import time
from IPython.display import clear_output

In [ ]:
def read(filename):
    start = open(filename,'r').read().replace('\n', '').replace(' ', '')
    goal = open('goal.txt','r').read().replace('\n', '').replace(' ', '')
    start_state = []
    goal_state = []
    for i in range(0,len(start)):
        start_state.append(int(start[i]))
    for i in range(0,len(goal)):
        goal_state.append(int(goal[i]))
    return start_state, goal_state

In [ ]:
class State:

    def __init__(self, state, pre_move = 'root'):
        self.state = np.array([state[:3], state[3:6], state[6:]])
        self.pre_move = pre_move
        self.FindZero()

    def __eq__(self, other):
        if isinstance(other, State):
            return np.array_equal(self.state, other.state)

    def FindZero(self):
        index = np.where(self.state == 0)
        coor = list(zip(index[0], index[1]))
        return list(coor[0])
    
    def ValidMove(self, move):
        if move == 'up':
            if self.FindZero()[0] != 0:
                return True
        elif move == 'down':
            if self.FindZero()[0] != 2:
                return True
        elif move == 'left':
            if self.FindZero()[1] != 0:
                return True
        elif move == 'right':
            if self.FindZero()[1] != 2:
                return True
        else:
            print('invalid move')
            return False
    
    def SavedMove(self, move):
        if self.ValidMove(move):
            saved = deepcopy(self)
            row = self.FindZero()[0]
            col = self.FindZero()[1]
            if move == 'up':
                self.state[row][col] = self.state[row-1][col]
                self.state[row-1][col] = 0
                self.pre_move = 'up'
            elif move == 'down':
                self.state[row][col] = self.state[row+1][col]
                self.state[row+1][col] = 0
                self.pre_move = 'down'
            elif move == 'left':
                self.state[row][col] = self.state[row][col-1]
                self.state[row][col-1] = 0
                self.pre_move = 'left'
            elif move == 'right':
                self.state[row][col] = self.state[row][col+1]
                self.state[row][col+1] = 0
                self.pre_move = 'right'
            return saved
        else:
            return False
        
    def Expand(self):
        nodes = []
        action_list = ['up','down','left','right']
        loop_move = ['down','up','right','left']
        for action in range(0,4):
            if(self.ValidMove(action_list[action]) and self.pre_move != loop_move[action]):
                temp = deepcopy(self)
                temp.SavedMove(action_list[action])
                nodes.append(temp)
        return nodes



In [ ]:
def Visited(state, state_list):
    for s in state_list:
        if s == state:
            return True
    return False

def DFS(state, goal, depth, limit):
    global visitedNode, n_expand, path

    if Visited(state, visitedNode):
        return False, False
    
    visitedNode.append(state)

    if depth == 0:
        if state == goal:
            path.append(state.pre_move)
            return True, False
            
    else:
        if n_expand >= limit:
            return False, True
        for node in state.Expand():
            n_expand +=1
            found, maxed = DFS(node, goal, depth-1, limit)
            if found:
                path.append(state.pre_move)
                return True, False
            if maxed:
                return False, True
    
    visitedNode.remove(state)
    return False, False

def ItertiveDFS(start, goal, limit):
    global n_expand
    depth = 0
    while True:
        n_expand = 0
        found, maxed = DFS(start, goal, depth, limit)

        clear_output(wait=True)
        print('Current nodes expanded', n_expand)
        print('Current depth',depth)
        time.sleep(0.01)
        
        if found:
            path.reverse()
            print(path)
            print('# moves:', len(path)-1)
            print('# nodes:', n_expand)
            break
        if maxed:
            print('Limit reached')
            break
        depth += 1
    print('Depth used =', depth)
    return

In [ ]:

visitedNode = []
path = []
n_expand = 0

def main():
    start_state, goal_state = read('input.txt')
    start = State(start_state)
    goal = State(goal_state)
    depth = ItertiveDFS(start, goal, 1000000)
    
main()
